In [1]:
import os.path as osp

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [49]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


#https://colab.research.google.com/drive/1N3LvAO0AXV4kBPbTMX866OwJM9YS6Ji2?usp=sharing#scrollTo=fl5W1gg5Jhzz

df_players = pd.read_csv("./../data_scrapped/atp_players.csv")
df_matchs = pd.read_csv("./../data_formatted/training_dgl_dataset.csv")

In [50]:
#On construit le graph

# lecture des données attendues
# tensor1[1] -> tensor2[1] 
# le player index en tensor1 a joué contre le player au meme index en tensor 2
# y stocke le résultat du match


tensor1=[]
tensor2=[]

for index,row in df_players.iterrows():
    winmatchs = df_matchs[df_matchs.player1_id == row.player_id]
    #print(len(winmatchs))
    if len(winmatchs) > 0:
        #print(row.player_id)
        for index2, row2 in winmatchs.iterrows():
                tensor1.append(index)
                tensor2.append(df_players.loc[df_players.player_id == row2.player2_id].index[0])

In [51]:
import numpy
import random

labels = numpy.ones(len(tensor1))

indexes = random.sample(range(0, len(tensor1)-1), ((len(tensor1)-1)//2))

for index in indexes:
    temp = tensor1[index]
    tensor1[index] = tensor2[index]
    tensor2[index] = temp
    labels[index] = 0

In [52]:
import torch_geometric.transforms as T
from torch_geometric.data import Data

# Create the heterogeneous graph data object:
#data = Data()

# Add the user nodes:
x = torch.tensor(list(df_players[["birth_year","weight_kg","height_cm"]].values),dtype=torch.float)  # [num_users, num_features_users]
x = torch.nan_to_num(x, nan=0.0)
#x = torch.masked_select(x, ~torch.isnan(x))
#x = torch.ones(df_players.shape[0])

edge_index = torch.stack([torch.tensor(tensor1), torch.tensor(tensor2)], dim=0)
#edge_attr = torch.Tensor(list(df_matchs[["player1_atprank","player1_oddsB365","player2_atprank","player2_oddsB365"]].values))
#y = torch.Tensor(list(df_matchs[["winner_player1"]].values))

labels = torch.tensor(labels, dtype=torch.long)

data = Data(x=x, edge_index=edge_index,edge_label=labels) 
#data = T.ToUndirected()(data)
# Add the movie nodes:
#data['movie'].x = movie_features  # [num_movies, num_features_movies]

data

Data(x=[3446, 3], edge_index=[2, 51981], edge_label=[51981])

In [12]:
labels

tensor([1., 0., 1.,  ..., 0., 1., 1.])

In [53]:
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.05,
    num_test=0.1,
    is_undirected=False,
    add_negative_train_samples=False,
    neg_sampling_ratio=0
)(data)
train_data, val_data, test_data

(Data(x=[3446, 3], edge_index=[2, 44184], edge_label=[44184], edge_label_index=[2, 44184]),
 Data(x=[3446, 3], edge_index=[2, 44184], edge_label=[2599], edge_label_index=[2, 2599]),
 Data(x=[3446, 3], edge_index=[2, 46783], edge_label=[5198], edge_label_index=[2, 5198]))

In [73]:
from torch_geometric.nn import SAGEConv,MLP

class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SAGEConv(3, 128)
        self.conv2 = SAGEConv(128, 64)
        self.mlp = MLP(in_channels=128, hidden_channels=128, out_channels=1, num_layers=3)

    def forward(self, x, edge_index, edge_label_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return self.mlp(torch.cat((x[edge_label_index[0]], x[edge_label_index[1]]), dim=-1))
    
model = Net().to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()


def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_label_index)
    loss = criterion(out.squeeze(), data.edge_label.float())
    loss.backward()
    optimizer.step()
    return loss


@torch.no_grad()
def test(data):
    model.eval()
    out = model(data.x, data.edge_index, data.edge_label_index)
    return roc_auc_score(data.edge_label.float(), out.squeeze())


best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train(train_data)
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, 'f'Test: {test_auc:.4f}')


Epoch: 001, Loss: 0.7175, Val: 0.5382, Test: 0.5300
Epoch: 002, Loss: 0.6960, Val: 0.5036, Test: 0.5092
Epoch: 003, Loss: 0.6931, Val: 0.5024, Test: 0.5069
Epoch: 004, Loss: 0.6927, Val: 0.5186, Test: 0.5156
Epoch: 005, Loss: 0.6899, Val: 0.5406, Test: 0.5307
Epoch: 006, Loss: 0.6886, Val: 0.5508, Test: 0.5411
Epoch: 007, Loss: 0.6874, Val: 0.5490, Test: 0.5411
Epoch: 008, Loss: 0.6871, Val: 0.5443, Test: 0.5307
Epoch: 009, Loss: 0.6856, Val: 0.5341, Test: 0.5191
Epoch: 010, Loss: 0.6858, Val: 0.5338, Test: 0.5178
Epoch: 011, Loss: 0.6857, Val: 0.5353, Test: 0.5166
Epoch: 012, Loss: 0.6846, Val: 0.5360, Test: 0.5186
Epoch: 013, Loss: 0.6846, Val: 0.5339, Test: 0.5162
Epoch: 014, Loss: 0.6848, Val: 0.5350, Test: 0.5171
Epoch: 015, Loss: 0.6840, Val: 0.5351, Test: 0.5165
Epoch: 016, Loss: 0.6838, Val: 0.5321, Test: 0.5157
Epoch: 017, Loss: 0.6839, Val: 0.5296, Test: 0.5144
Epoch: 018, Loss: 0.6834, Val: 0.5411, Test: 0.5219
Epoch: 019, Loss: 0.6832, Val: 0.5403, Test: 0.5213
Epoch: 020, 

In [71]:
# Specify the edge you want to predict (e.g., edge from node 0 to node 1)
node_0 = 3
node_1 = 664

print(df_players.iloc[node_0])
#print(df_players.iloc[node_0]["first_name","last_name"])

# Predict the direction of the edge
with torch.no_grad():
    model.eval()
    edge_label_index =  torch.tensor([[node_0, node_1], [node_1, node_0]], dtype=torch.long).t()
    print(edge_label_index)
    prediction = model(data.x, data.edge_index, edge_label_index).view(-1).sigmoid()
    print(prediction)

player_id                l836
first_name               Nick
last_name             Lindahl
country_code              AUS
birthdate          1988.07.31
birth_year             1988.0
birth_month               7.0
birth_day                31.0
turned_pro             2006.0
weight_kg                77.0
weight_lbs              170.0
height_cm               183.0
height_in                72.0
birthplace      Malmo, Sweden
Name: 3, dtype: object
tensor([[  3, 664],
        [664,   3]])
tensor([0.9444, 0.9608])
